In [ ]:
from google.colab import drive
drive.mount('/content/gdriver')

In [ ]:
# Các thư viện sử dụng
import time
from matplotlib import pyplot as plt
import imutils
import dlib
import cv2
import pickle
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications
import tensorflow as tf
from keras import backend as K
import h5py
from tqdm import tqdm
import glob
import os

In [ ]:
# Hàm nhận đường dẫn của ảnh và trả về ảnh kích thước(221x221) chứa khuôn mặt trong ảnh.

def read_image(image_path):
    img = cv2.imread(image_path)
    # grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    hog_face_detector = dlib.get_frontal_face_detector()
    faces_hog= hog_face_detector(img, 1)
    
    if len(faces_hog) == 1:
        x = faces_hog[0].left()
        y = faces_hog[0].top()
        w = faces_hog[0].right() - x
        h = faces_hog[0].bottom() - y
    
        img = cv2.resize(img[y:y+h, x:x+w], (221, 221))
        return img
        
    return None

In [ ]:
# Đưa toàn bộ ảnh Dataset vào hàm read_image và lưu ảnh khuôn mặt vào images[], labels=[0,1], names=[CR7, M10]

images = []
labels = []
names = []
count = 0

for path, dirs, files in os.walk('/content/gdriver/My Drive/FaceRecognition_KNN/Dataset'):
        for d in tqdm(dirs):
            class_image = []
            for ext in ('jpg', 'jpeg', 'png'):
                for f in glob.glob(os.path.join(path, d, '*.' + ext)):
                    try:
                        img = read_image(f)
                    except:
                        continue
                    if img is None:
                        continue
                    class_image.append(img)
                    
            if len(class_image) > 1:
                names.append(d)
                for img in class_image:
                    images.append(img)
                    labels.append(count)
                count += 1

G=np.size(labels)

In [ ]:
# Mạng học sâu ConvNet

def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model
deep_rank_model = deep_rank_model()
deep_rank_model.summary()

In [ ]:
# Biến toàn bộ ảnh khuôn mặt trong images[] thành vector 128 rồi lưu vào ma trận embs128[]
from tqdm import tqdm
embs128 = []
for i in range(G):
    x= images[i]
    image = x/255.
    image = np.expand_dims(image, axis=0)
    emb128 = deep_rank_model.predict([image, image, image])
    embs128.append(emb128)

In [ ]:
# Hàm khoảng cách distance và hàm tính knn

def distance(v1, v2): 
	return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, label, k):
	dist = []
	
	for i in range(100):
		# Get the vector and label
		ix = train[i]
		iy = label[i]
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
		
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0]) [:k]
	so=0
	for j in range(k):
		so+= dk[j][1]
	so= so/k
	if so > 0.5:
		return 1
	else:
		 return 0

In [ ]:
from time import time
# Nhập giá trị K
K= 2

# Tính Accuracy test và Time train cho toàn bộ tập test

t0= time()
dem=0
for path, dirs, files in os.walk('/content/gdriver/My Drive/FaceRecognition_KNN/Test'):
    for h in tqdm(dirs):
      # class_image = []
      for f in glob.glob(os.path.join(path, h, '*.' + 'jpg')):
        img1= read_image(f)
        img1= np.array(img1)
        img1= img1/255
        img1 = np.expand_dims(img1, axis=0)
        emb = deep_rank_model.predict([img1, img1, img1])
        out= knn(embs128,emb,labels,K)
        if names[out] == h:
          dem+=1
t1= time()

In [ ]:
def output(K, dem, t0, t1):
  # Giá trị K
  print("K = ", K)
  # Accuracy test
  print(dem*100/90)
  # Time train
  print(t1 - t0)
output(K, dem ,t0, t1)

In [ ]:
# Ghi ra tên mà chương trình dự đoán với một ảnh bất kì
img1= read_image('/content/gdriver/My Drive/FaceRecognition_KNN/Test/M10/21.jpg')
img1= np.array(img1)
img1= img1/255
img1 = np.expand_dims(img1, axis=0)
emb = deep_rank_model.predict([img1, img1, img1])
np.shape(emb)
out= knn(embs128,emb,labels,3)
print(names[out])